In [187]:
import subprocess
# cmd0 = "sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable"
# print(os.system(cmd0))
# cmd1 = "sudo apt-get update"
# print(os.system(cmd1))
# cmd2 = "sudo apt-get install libgdal-dev"
# print(os.system(cmd2))
# cmd3 = "virtualenv gdalenv"
# print(os.system(cmd3))
# cmd4 = "source gdal/bin/activate"
# print(os.system(cmd4))
# cmd5 = "pip install --no-install GDAL"
# print(os.system(cmd5))
# cmd6 = "python setup.py build_ext --include-dirs=/usr/include/gdal/"
# print(os.system(cmd6))
# cmd7 = "pip install gdal --global-option=build_ext --global-option='-I/usr/include/gdal/'"
# print(os.system(cmd7))

# cmd0 = "pip install GDAL==$(gdal-config --version) --global-option=build_ext --global-option='-I/usr/include/gdal' "
cmd0 = "conda install gdal"
print(os.system(cmd0))
print(subprocess.check_output(["gdalinfo", "--version"]))
# from osgeo import gdal
# import gdal

0
b'GDAL 3.0.4, released 2020/01/28\n'


In [188]:
import numpy as np
import os
import shutil
import urllib.request as urllib
import zipfile
import subprocess
import sys

from PIL import Image

#### Define the version number of your algorithm. Consider using [Semantic Versioning](https://semver.org/)

In [189]:
VERSION = 'latest'

#### Provide information on the creator and contributors of this algorithm

In [190]:
ALGORITHM_AUTHOR = 'Unknown'
ALGORITHM_AUTHOR_EMAIL = 'author@example.com'
ALGORITHM_CONTRIBUTORS = [""]

#### Name and describe your algorithm

In [191]:
ALGORITHM_NAME = 'default'
ALGORITHM_DESCRIPTION = ''

#### Provide citation information for algorithm publication. This includes the citation author, the citation title, and the citation year

In [192]:
CITATION_AUTHOR = ''
CITATION_TITLE = ''
CITATION_YEAR = ''

#### Include the name(s) of the variable(s) used in the algorithm, separated by commas

In [193]:
VARIABLE_NAMES = 'size of image channels'

#### Include the units and labels of the variables, matching the order of VARIABLE_NAMES, also separated by commas. VARIABLE_LABELS is an optional field and can be left empty.

In [194]:
VARIABLE_UNITS = 'pixels'
VARIABLE_LABELS = ''

#### Optional override for the generation of a BETYdb compatible csv file. Set to False to suppress the creation of a compatible file

In [195]:
WRITE_BETYDB_CSV = True

#### Optional override for the generation of a TERRA REF Geostreams compatible csv file. Set the variable to False to suppress the creation of a compatible file

In [196]:
WRITE_GEOSTREAMS_CSV = True

#### Change the url below to get images from a different source. This assumes that the image files are contained within a .zip file. The code below will add the sample_plot_images chosen to the system

In [197]:
url = "https://de.cyverse.org/dl/d/4108BB75-AAA3-48E1-BBD4-E10B06CADF54/sample_plot_images.zip"

if os.path.isdir("sample_plot_images"):
    shutil.rmtree("sample_plot_images")
os.mkdir("sample_plot_images")
if not os.path.isfile("sample_plot_images.zip"):
    urllib.urlretrieve(url, "sample_plot_images.zip")
zipfile.ZipFile("sample_plot_images.zip", 'r').extractall("sample_plot_images")
os.remove("sample_plot_images.zip")

#### Define your calculate() function

In [198]:
def calculate(pxarray: np.ndarray):
    """Calculates one or more values from plot-level RGB data
    Arguments:
        pxarray: Array of RGB data for a single plot
    Return:
        Returns one or more calculated values
    """
    # ALGORITHM: replace the following lines with your algorithm
    channel_size = pxarray[:, :, 1].size

    # RETURN: replace the following return with your calculated values. Be sure to order them as defined in VARIABLE_NAMES above
    return channel_size

#### Test the calculate() function on the sample plot images located in the sample_plot_images folder

In [199]:
for filename in os.listdir("sample_plot_images"):
    img = Image.open("sample_plot_images/" + filename)
    img_arr = np.array(img)
    print(calculate(img_arr))

7000
7000
7000
7000
7000
7000


#### Next Generate your Dockerfile by running the generate.py script

In [200]:
cmd0 = "python {{cookiecutter._project_name}}/generate.py"
os.system(cmd0)
cmd1 = "mv Dockerfile {{cookiecutter._project_name}}/Dockerfile"
os.system(cmd1)
cmd2 = "mv sample_plot_images {{cookiecutter._project_name}}/sample_plot_images"

#### Now build the dockerfile (Currently this will have a default project name and project version)

In [201]:
cmd = "docker build -t " + ALGORITHM_NAME + ":" + VERSION + " ."
os.system(cmd)

256

#### Next run the dockerfile for testing

In [202]:
cmd = 'docker run --rm --mount "src=`pwd`,target=/mnt,type=bind" default:latest --working_space "/mnt/{{cookiecutter.project_name}}/sample_plot_images"'
os.system(cmd)

0

### The following test whether the fields and algorithm are in the correct format

In [203]:
def _get_variables_header_fields() -> str:
    """Returns a string representing the variable header fields
    Return:
        Returns a string representing the variables' header fields
    """
    variables = VARIABLE_NAMES.split(',')
    labels = VARIABLE_LABELS.split(',')
    labels_len = len(labels)
    units = VARIABLE_UNITS.split(',')
    units_len = len(units)

    if labels_len != len(variables):
        sys.stderr.write("The number of defined labels doesn't match the number of defined variables")
        sys.stderr.write("  continuing processing")
    if units_len != len(variables):
        sys.stderr.write("The number of defined units doesn't match the number of defined variables")
        sys.stderr.write("  continuing processing")

    headers = ''
    for idx, variable_name in enumerate(variables):
        variable_header = variable_name
        if idx < labels_len:
            variable_header += ' - %s' % labels[idx]
        if idx < units_len:
            variable_header += ' (%s)' % units[idx]
        headers += variable_header + ','

    return headers

In [204]:
_get_variables_header_fields()

'size of image channels -  (pixels),'

In [205]:
def print_usage():
    """Displays information on how to use this script
    """
    argc = len(sys.argv)
    if argc:
        our_name = os.path.basename(sys.argv[0])
    else:
        our_name = os.path.basename(__file__)
    print(our_name + " <folder>|<filename> ...")
    print("    folder:   path to folder containing images to process")
    print("    filename: path to an image file to process")
    print("")
    print("  One or more folders and/or filenames can be used")
    print("  Only files at the top level of a folder are processed")

In [206]:
print_usage()

ipykernel_launcher.py <folder>|<filename> ...
    folder:   path to folder containing images to process
    filename: path to an image file to process

  One or more folders and/or filenames can be used
  Only files at the top level of a folder are processed


In [207]:
def check_configuration():
    """Checks if the configuration is setup properly for testing
    """
    if not VARIABLE_NAMES:
        sys.stderr.write("Variable names configuration variable is not defined yet. Please define and try again")
        sys.stderr.write("    Set VARIBLE_NAMES variable with your variable names")
        return False

    return True

In [208]:
check_configuration()

True

In [209]:
def run_test(filename):
    """Runs the extractor code using pixels from the file
    Args:
        filename(str): Path to image file
    Return:
        The result of calling the extractor's calculate() method
    Notes:
        Assumes the path passed in is valid. An error is reported if
        the file is not an image file.
    """
    try:
        open_file = gdal.Open(filename)
        if open_file:
            # Get the pixels and call the calculation
            pix = np.array(open_file.ReadAsArray())
            calc_val = calculate(np.rollaxis(pix, 0, 3))

            # Check for unsupported types
            if isinstance(calc_val, set):
                raise RuntimeError("A 'set' type of data was returned and isn't supported.  Please use a list or a tuple instead")

            # Perform any type conversions to a printable string
            if isinstance(calc_val, str):
                print_val = calc_val
            else:
                # Check if the return is iterable and comma separate the values if it is
                try:
                    _ = iter(calc_val)
                    print_val = ",".join(map(str, calc_val))
                except Exception:
                    print_val = str(calc_val)

            print(filename + "," + print_val)
    except Exception as ex:
        sys.stderr.write("Exception caught: " + str(ex) + "\n")
        sys.stderr.write("    File: " + filename + "\n")

In [210]:
for filename in os.listdir("sample_plot_images"):
    run_test("sample_plot_images/" + filename)

Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_17_7_W.tif
Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_40_11_W.tif
Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_6_1_E.tif
Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_1_2_E.tif
Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_33_8_W.tif
Exception caught: name 'gdal' is not defined
    File: sample_plot_images/rgb_5_11_W.tif
